## Adata single test code

In this part we show the analysis we ran on each single group to have look at the expression in each sample to identfiy the cell subgroups. 

Not to relevant. 

In [ ]:
adata_basal = sc.read_h5ad("/omics/odcf/analysis/OE0228_projects/VascularAging/rna_sequencing/public_scrnaseq/lung_lps/basal/anndata/adata_norm_processed.h5ad")

In [ ]:
f_paths=[f"/omics/odcf/analysis/OE0228_projects/VascularAging/rna_sequencing/public_scrnaseq/lung_lps/{s_name}/anndata/adata_norm_processed.h5ad"for s_name in sample_names]



In [ ]:
adata_dict = dict(zip(sample_names, adata))

In [ ]:
samp_name = "Day2"
check_pro_ecs = False

adata_obj = adata_dict[samp_name]
adata_obj.obs["Clusters"] = labels[samp_name]["Clusters"]
adata_obj.obs["Clusters"][adata_obj.obs["Clusters"].isnull()] = 99
adata_obj.obs["Clusters"] = adata_obj.obs["Clusters"].astype("category")
#sc.pp.neighbors(adata_obj,10)
#sc.tl.umap(adata_obj)
sc.pl.umap(
    adata_obj,
    color="Clusters",  
)

In [ ]:
adata_obj.obs["Clusters"]

In [ ]:
marker_genes = {"immuneEC": ["Cd74", "H2-Aa", "H2-Ab1", "Irf7", "Icam1", "Il4ra"], 
                "devEC": ["Junb", "Egr1", "Sox17", "Kdr", "Ace"], 
                "Aerocyte": ["Car4", "Ednrb", "Emp2", "Itgb5"],
             }

if check_pro_ecs:
      marker_genes["proEC"] = ["Cdk1", "Tk1", "E2f1", "Ccna2", "Ccne1"]

adata_obj.layers["counts"] = adata_obj.X
adata_obj.X = adata_obj.layers["scran_normalization"]
adata_obj.var["highly_variable"] = adata_obj.var["highly_deviant"]
sc.tl.pca(adata_obj, n_comps=50, use_highly_variable=True)
sc.pp.neighbors(adata_obj)
sc.tl.umap(adata_obj)

In [ ]:
sc.pl.umap(adata_obj,color = "Clusters")
for ct in marker_genes.keys():
    print(f"{ct.upper()}:")  # print cell subtype name
    sc.pl.umap(
        adata_obj,
        color=marker_genes[ct],
        vmin=0,
        vmax="p99",  # set vmax to the 99th percentile of the gene count instead of the maximum, to prevent outliers from making expression in other cells invisible. Note that this can cause problems for extremely lowly expressed genes.
        sort_order=False,  # do not plot highest expression on top, to not get a biased view of the mean expression among cells
        frameon=False,
        cmap="Reds",  # or choose another color map e.g. from here: https://matplotlib.org/stable/tutorials/colors/colormaps.html
    )
    print("\n\n\n")


    
sc.pl.umap(
        adata_obj,
        color=["Aplnr", "Apln"],
        vmin=0,
        vmax="p99",  # set vmax to the 99th percentile of the gene count instead of the maximum, to prevent outliers from making expression in other cells invisible. Note that this can cause problems for extremely lowly expressed genes.
        sort_order=False,  # do not plot highest expression on top, to not get a biased view of the mean expression among cells
        frameon=False,
        cmap="Reds",  # or choose another color map e.g. from here: https://matplotlib.org/stable/tutorials/colors/colormaps.html
    )   

The figure above shows the expression of Marker genes for AEROCYTES IMMUNEC AND DEVEC groups. 
Aplnr and Apln are also added. 



In [ ]:
#sc.tl.louvain(adata_obj,resolution=0.4,)
sc.tl.leiden(adata_obj,resolution=0.4, key_added="leiden_res0_4")
sc.pl.umap(
    adata_obj,
    color=["leiden_res0_25", "leiden_res0_5","leiden_res0_4", "leiden_res1"], ncols = 2
)

Test of various clustering settings -> However we used clustering provided by the authors


In [ ]:
sc.tl.rank_genes_groups(
    adata_obj, groupby="Clusters", method="wilcoxon", key_added = "dea_clusters"
)

In [ ]:
sc.pl.umap(
    adata_obj,
    color=["Aplnr"], ncols = 2
)

In [ ]:
 adata_obj.obs["Clusters"] == "5"
    

In [ ]:
sc.pl.rank_genes_groups_dotplot(
    adata_obj, groupby="Clusters", standard_scale="var", n_genes=5, key="dea_clusters", dendrogram = False)


Top Differentially expressed genes for the cluster groups + the three subgroup. 
Group 99 are cells not present in the clustering provided. 

In [ ]:
sc.tl.filter_rank_genes_groups(
    adata_obj,
    min_in_group_fraction=0.2,
    max_out_group_fraction=0.2,
    key="dea_clusters",
    key_added="dea_clusters_filtered",
)


In [ ]:
sc.pl.rank_genes_groups_dotplot(
    adata_obj,
    groups = ['Aerocyte', 'devEC', 'immunec'],
    groupby="Clusters",
    standard_scale="var",
    n_genes=15,
    key="dea_clusters_filtered",
    dendrogram = False
)


After filtering celar marker genes barely present, since they are not really unique. 

In [ ]:
marker_genes = {"immuneEC": ["Cd74", "H2-Aa", "H2-Ab1"], "devEC": ["Junb", "Egr1", "Sox17"], "Aerocyte": ["Car4", "Ednrb", "Emp2", "Itgb5"]}

adata_basal.layers["counts"] = adata_basal.X
adata_basal.X = adata_basal.layers["scran_normalization"]
adata_basal.var["highly_variable"] = adata_basal.var["highly_deviant"]
sc.tl.pca(adata_basal, n_comps=50, use_highly_variable=True)
sc.pp.neighbors(adata_basal)
sc.tl.umap(adata_basal)

In [ ]:
for ct in marker_genes.keys():
    print(f"{ct.upper()}:")  # print cell subtype name
    sc.pl.umap(
        adata_basal,
        color=marker_genes[ct],
        vmin=0,
        vmax="p99",  # set vmax to the 99th percentile of the gene count instead of the maximum, to prevent outliers from making expression in other cells invisible. Note that this can cause problems for extremely lowly expressed genes.
        sort_order=False,  # do not plot highest expression on top, to not get a biased view of the mean expression among cells
        frameon=False,
        cmap="Reds",  # or choose another color map e.g. from here: https://matplotlib.org/stable/tutorials/colors/colormaps.html
    )
    print("\n\n\n")

In [ ]:
dc.run_ora(
    mat=adata_obj,
    net=pd.concat([hallmarks, full_sen]),
    source='geneset',
    target='genesymbol',
    verbose=True,
    use_raw=False
)



In [ ]:
acts = dc.get_acts(adata_obj, obsm_key='ora_estimate')

# We need to remove inf and set them to the maximum value observed
acts_v = acts.X.ravel()
max_e = np.nanmax(acts_v[np.isfinite(acts_v)])
acts.X[~np.isfinite(acts.X)] = max_e

# We can scale the obtained activities for better visualizations
sc.pp.scale(acts)
acts



In [ ]:
sc.pl.umap(acts, color=['HALLMARK_DNA_REPAIR', 'HALLMARK_INTERFERON_ALPHA_RESPONSE', 
                        'FRIDMAN_SENESCENCE_DN', 'FRIDMAN_SENESCENCE_UP', 'ec_Senescence_sig'], cmap='RdBu_r', vcenter=0)
sc.pl.violin(acts, keys=['HALLMARK_DNA_REPAIR', 'HALLMARK_INTERFERON_ALPHA_RESPONSE', 
                        'FRIDMAN_SENESCENCE_DN', 'FRIDMAN_SENESCENCE_UP', 'ec_Senescence_sig'], groupby='leiden_res0_25', rotation=90)



In [ ]:
adata